## Interactive plots demo using NASA Exoplanet Archive

#### This tutorial will show you how to plot up an interactive exoplanet mass-radius diagram using bokeh or altair.

First we'll grab some data about known (confirmed) exoplanets from the [NASA Exoplanet Archive](https://exoplanetarchive.ipac.caltech.edu/index.html) using astroquery:

In [33]:
from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive
from astropy.table import Table
exoplanet_archive_table = NasaExoplanetArchive.get_confirmed_planets_table()
exoplanet_archive_table = Table(exoplanet_archive_table)

In [46]:
exoplanet_archive_table.info

<Table masked=True length=3708>
      name       dtype      unit    n_bad
--------------- ------- ----------- -----
    pl_hostname   str27                 0
      pl_letter    str1                 0
  pl_discmethod   str29                 0
        pl_pnum   int64                 0
      pl_orbper float64           d     0
  pl_orbpererr1 float64           d     0
  pl_orbpererr2 float64           d     0
   pl_orbperlim float64           d     0
     pl_orbpern   int64                 0
     pl_orbsmax float64          AU     0
 pl_orbsmaxerr1 float64          AU     0
 pl_orbsmaxerr2 float64          AU     0
  pl_orbsmaxlim float64          AU     0
    pl_orbsmaxn   int64                 0
    pl_orbeccen float64              2536
pl_orbeccenerr1 float64              2871
pl_orbeccenerr2 float64              2871
 pl_orbeccenlim   int64              2489
   pl_orbeccenn   int64                 0
     pl_orbincl float64         deg     0
 pl_orbinclerr1 float64         deg     0
 p

### Making an interactive mass-radius diagram with bokeh

Filter out all the planets with no known mass/radius, which are denoted with zeroes in the table:

In [44]:
filter = (exoplanet_archive_table['pl_bmassj'] > 0.0) \
        & (exoplanet_archive_table['pl_radj'] > 0.0)

In [61]:
print("There are {0} confirmed planets with measured mass + radius.".format(
    len(exoplanet_archive_table[filter])))

There are 600 confirmed planets with measured mass + radius.


Now load up bokeh:

In [53]:
from bokeh.plotting import ColumnDataSource, figure, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

First we have to make the data source. Fortunately, this is easy from an astropy table -- all we have to do is make it a pandas dataframe with `.to_pandas()` and bokeh can handle the rest. We'll do that using the filtered table.

In [54]:
source = ColumnDataSource(exoplanet_archive_table[filter].to_pandas())

The first step is to set up the figure characteristics. We'll enable a few basic interactive functionalities here, as well as setting the axis ranges.

In [55]:
fig = figure(tools="pan,wheel_zoom,box_zoom,reset", active_scroll="wheel_zoom",
            x_range=[0.,10.], y_range=[0.,3.]) 

Now we can add features to the plot using the column names in the data source. To see what the figure looks like at any point, just do `show(fig)`.

In [62]:
pl_render = fig.circle('pl_bmassj','pl_radj', source=source, size=10)
show(fig)

Not bad! We can scroll around and zoom in and out. But what if we want more functionality? Let's add some tooltips to tell us more about the planets on hover.

In [63]:
from bokeh.models import HoverTool
hover = HoverTool(renderers=[pl_render],
                    tooltips=[
        ("name", "@NAME_LOWERCASE"),
        ("mass", "@pl_bmassj"),
        ("radius", "@pl_radj"),
        ("discovered by", "@pl_discmethod")
        ]
    )
fig.add_tools(hover)

In [64]:
show(fig)

One issue here is that we haven't included any uncertainty estimates. These are pretty important, especially once we get down to the smallest planets. 

Let's render some error bars and make the point opacity scale with the inverse variance. This will draw the viewer's eye to the best-measured planets.

If you want to save this plot to disk as an html file, you can do so like this:

In [67]:
from bokeh.plotting import output_file, save
output_file('bokeh_massradius.html')
save(fig)

'/Users/mbedell/python/astro-dataviz/bokeh_massradius.html'

### Linking interactive plots with altair